In [16]:
import pandas as pd
import numpy as np

In [17]:
!git clone https://github.com/9berny9/bikes

fatal: destination path 'bikes' already exists and is not an empty directory.


# BIKES IN EDINBURGH
1. [Zadání](#1)
2. [Úprava dat](#2)
3. [Analýza](#3)



## Zadání <a name="1"></a>



V Edinburghu, stejně jako v dalších městech, funguje systém "bike sharing" - ve městě jsou stanice s koly, člověk si může nějaké půjčit a potom ho vrátit v nějaké další stanici. Problém je, že v některých stanicích se kola pravidelně hromadí a jinde naopak chybí. Provozovatel kol, firma Just Eat Cycles, zadala projekt, jehož cílem je systém zefektivnit.

Coby datový analytik jste součástí týmu, který na projektu pracuje. Vaším úkolem je zpracovat relevantní data a zjistit z nich informace užitečné pro zbytek týmu. Máte k dispozici data o všech výpůjčkách (na ENGETO databázi v tabulce edinburgh_bikes). Proveďte standardní deskriptivní statistiku dat. Také zjistěte minimálně následující informace:

identifikujte aktivní a neaktivní stanice
identifikujte nejfrekventovanější stanice
identifikujte stanice, na kterých se kola hromadí a stanice, kde potenciálně chybí
spočítejte vzdálenosti mezi jednotlivými stanicemi
jak dlouho trvá jedna výpůjčka? Najděte odlehlé hodnoty, zobrazte histogram
Analýza poptávky:

zobrazte vývoj poptávky po půjčování kol v čase
identifikujte příčiny výkyvů poptávky
zjistěte vliv počasí na poptávku po kolech (údaje o počasí v Edinburghu jsou v tabulce edinburgh_weather)
půjčují si lidé kola více o víkendu než během pracovního týdne?
Výstupem analýzy bude ideálně jupyter notebook nebo colab notebook uložený na Vašem githubu. Výstupy Vaší analýzy musí být zřejmé i bez spouštění kódu. V textových buňkách popište, jak jste postupovali a napište své závěry. Pokud v zadání chybí nějaká informace, doplňte ji podle vlastního uvážení, své rozhodnutí uveďte a vysvětlete v notebooku.

## Úprava dat <a name="2"></a>

* Veškerá práce s databází a úprava datových formátů se nachází v repozitáři zde: https://github.com/9berny9/bikes/blob/90970fb641920d89a08362277943fd1b273625eb/data_process.ipynb

* Výstupem úpravy dat jsou pickle soubory uložené v repozitáři **bikes/Database/modified_data**, na kterých probíhá celá analýza.

## Analýza<a name="3"></a>

### Vzdálenosti stanic<a name="3.1"></a>

In [18]:
from sklearn.metrics import DistanceMetric

In [19]:
url_stations = "/content/bikes/Database/modified_data/stations.pkl"
df = pd.read_pickle(url_stations)

In [20]:
df = df.sort_values(by="id")

https://medium.com/@danalindquist/finding-the-distance-between-two-lists-of-geographic-coordinates-9ace7e43bb2f

In [21]:
# převedení na radiány
df['lat'] = np.radians(df['lat'])
df['lon'] = np.radians(df['lon'])

In [22]:
# vytvoření matice pomocí knihovny
dist = DistanceMetric.get_metric('haversine')

In [23]:
# radius země v km
earth_radius = 6373
distance = pd.DataFrame(dist.pairwise((df[['lat','lon']]) * earth_radius).round(2),  columns=df.id, index=df.id)

In [24]:
distance

id,171,183,189,225,241,242,246,247,248,249,...,1868,1869,1870,1871,1874,1877,2259,2263,2265,2268
id,,,,,,,,,,,,,,,,,,,,,
171,0.00,0.96,0.80,1.03,1.84,1.84,0.64,1.92,0.33,1.15,...,1.07,0.89,0.23,1.09,1.30,0.70,2.47,0.58,1.05,1.51
183,0.96,0.00,0.19,0.81,1.13,1.13,1.59,1.75,0.66,1.78,...,1.58,1.60,1.18,0.57,1.36,1.59,1.51,1.37,1.95,0.80
189,0.80,0.19,0.00,0.70,1.18,1.18,1.44,1.88,0.48,1.74,...,1.56,1.53,1.02,0.69,1.43,1.46,1.67,1.18,1.76,0.83
225,1.03,0.81,0.70,0.00,0.83,0.83,1.51,2.54,0.79,2.18,...,2.09,1.92,1.22,1.37,2.11,1.69,1.65,0.97,1.47,0.58
241,1.84,1.13,1.18,0.83,0.00,0.00,2.33,2.10,1.54,2.91,...,2.69,2.70,2.04,1.54,2.28,2.52,0.85,1.76,2.08,0.35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1877,0.70,1.59,1.46,1.69,2.52,2.52,0.26,1.53,1.02,0.53,...,0.59,0.32,0.49,1.42,1.05,0.00,2.81,0.82,0.78,2.21
2259,2.47,1.51,1.67,1.65,0.85,0.85,3.07,1.30,2.16,2.32,...,2.23,2.50,2.68,1.51,1.77,2.81,0.00,2.60,2.65,1.08
2263,0.58,1.37,1.18,0.97,1.76,1.75,0.58,2.34,0.75,1.35,...,1.39,1.13,0.54,1.66,1.78,0.82,2.60,0.00,0.59,1.55


In [25]:
distance.max().sort_values(ascending=False)

id
1723    3.13
1767    3.13
225     3.13
1819    3.13
290     3.13
        ... 
1731    2.79
2268    2.78
965     2.77
1057    2.73
1027    2.71
Length: 200, dtype: float64